This model will read data from a large set and Classify the Sentiment:
Positive, Negative, Natural

In [4]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
data = pd.read_csv('swiggy.csv')
data.head()


In [ ]:
# Small code snippet for Tokenizer:

sentences = [
    'I am a good Boy', 
    'This is a good desk',
    'Boy is roaming around'
]
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(sentences)
